In [1]:
import stratigraph as sg
from mayavi import mlab
import matplotlib.pyplot as plt
import numpy as np
import h5py
from tqdm import trange
import scipy.io as sio
from PIL import ImageFont
from PIL import ImageDraw 
import matplotlib as mpl
from matplotlib.colors import ListedColormap
from matplotlib import gridspec

# set up graphics:
%matplotlib qt
plt.rcParams['svg.fonttype'] = 'none'

## TDWB-17-1
### Load data

The TDWB-17-1 surfaces are available from [this Dropbox location](https://www.dropbox.com/scl/fi/r5goh7o902pv2rkhsqkea/Zdata_TDWB_17_1.mat?rlkey=jt34a5mvuzd6rblq2ylcva0ke&dl=0). The original data, including many other items in addition to the surfaces, can be found here:

[Straub, K.M., 2019, TDWB_17_1, SEAD, DOI: 10.26009/S0FCQNQR](http://sead-published.ncsa.illinois.edu/seadrepository/api/researchobjects/urn:uuid:5d628abfe4b011c83f66a7af).

In [78]:
tdwb_17 = sio.loadmat('/Users/zoltan/Dropbox/Chronostratigraphy/Tulane/Zdata_TDWB_17_1.mat')
tdwb_17.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Zdata'])

In [180]:
import pandas as pd
df = pd.read_csv('../data/TDWB_17_1_Subside.csv')
df.head()

,Date,Time,RunTime (days),Run/Paused,OcnZ,TargetOcnZ,WeirZ,Qsed,Qin,Qin On/Off,Qaux On/Off,Unnamed: 11,Unnamed: 12
0,7/6/17,9:02:56,0.00000,R,-1157.1,-1160.0,-1.0,166.797,2.818,24,ON,OFF,NaN
1,7/6/17,9:03:57,0.00071,R,-1156.8,-1160.0,-1.0,265.494,2.818,24,ON,OFF,NaN
2,7/6/17,9:04:49,0.00131,P,-1156.1,-1160.0,-1.0,256.590,2.818,24,OFF,OFF,NaN
3,7/6/17,9:06:37,0.00133,P,-1156.9,-1160.0,-1.0,147.897,2.818,24,OFF,OFF,NaN
4,7/6/17,9:06:42,0.00133,P,-1156.9,-1160.0,-1.0,151.500,2.818,24,OFF,OFF,NaN


### Preprocess data

In [80]:
topo = tdwb_17['Zdata'].copy()
topo[topo > -1050] = -1050
topo[np.isnan(topo) == 1] = -1050

# adjust topographic and subsidence surfaces in the proximal corners (for aestethic reasons only)
inds = np.indices(np.shape(topo[:,:,0]))
inds2 = np.argwhere(inds[0] < -0.65*inds[1] + 148)
inds3 = np.argwhere(inds[0] > 0.58*inds[1] + 360)
for i in range(np.shape(topo)[2]):
    topo[:, :, i][inds2[:,0], inds2[:,1]] = -1075
    topo[:, :, i][inds3[:,0], inds3[:,1]] = -1075
    
# smooth the topographic surfaces a bit
for i in range(topo.shape[2]):
    topo[:,:,i] = sg.sgolay2d(topo[:,:,i], 5, 3)

In [81]:
scan_times  = np.arange(0, np.max(df['RunTime (days)'])+0.1, 0.125) # scan times in days

days = df['RunTime (days)'].values
sl = df['TargetOcnZ'].values

inds = np.hstack((0, np.where(np.diff(days) > 0)[0]+1))

days = days[inds]
sl = sl[inds]

from scipy.interpolate import interp1d

# Create an interp1d object
f = interp1d(days, sl, kind='cubic')

# Get the y values at some new values of x
sea_level = f(scan_times)

# Plot the data and the interpolated function
plt.figure()
plt.plot(df['RunTime (days)'], df['TargetOcnZ'])
plt.plot(scan_times, sea_level, 'o')

### Figure 1: Time-elevation (Barrell) plot example

In [125]:
row = 251
col = 47
res = 0.5
elevation = topo[row, col, :].copy()
fig, dve_data, duration_thickness_data, ts_labels, strat_tops, strat_top_inds, bound_inds, interval_labels \
      = sg.plot_strat_diagram(elevation, 'mm', scan_times, 'days', res, 
            np.max(elevation), np.max(scan_times), plotting=True, plot_raw_data=True)

In [128]:
# create animation
for i in range(2, len(elevation)+1):
    fig, dve_data, duration_thickness_data, ts_labels, strat_tops, strat_top_inds, bound_inds, interval_labels \
            = sg.plot_strat_diagram(elevation[:i], 'mm', scan_times[:i], 'days', res, np.max(elevation), 
                            np.max(scan_times), plotting=True, plot_raw_data=True)
    fname = '/Users/zoltan/Dropbox/Chronostratigraphy/movie_frames/TDWB_17_Barrell_plot_1_' + '%03d.png'%(i-1)
    fig.savefig(fname, dpi=300)
    plt.close('all')

In [162]:
# create animation
for i in range(10, 923, 10):
    elevation = topo[row, i, :].copy()
    fig, dve_data, duration_thickness_data, ts_labels, strat_tops, strat_top_inds, bound_inds, interval_labels \
            = sg.plot_strat_diagram(elevation, 'mm', scan_times, 'days', res, np.min(elevation), np.min(elevation)+200, 
                            np.max(scan_times), plotting=True, plot_raw_data=True)
    fname = '/Users/zoltan/Dropbox/Chronostratigraphy/movie_frames/TDWB_17_Barrell_plot_step_through_' + '%03d.png'%(i//10)
    fig.savefig(fname, dpi=300)
    plt.close('all')

In [178]:
for i in range(10, 923, 10):
    fig = plt.figure()
    plt.imshow(topo[:,:,-1], cmap = cmocean.cm.deep_r)
    plt.contour(topo[:,:,-1], colors='k', levels=100, linestyles='solid', linewidths=0.5)
    plt.xticks([])
    plt.yticks([])
    plt.plot(i, row, 'ro', markersize=10)
    fname = '/Users/zoltan/Dropbox/Chronostratigraphy/movie_frames/TDWB_17_Barrell_plot_step_through_map_' + '%03d.png'%(i//10)
    fig.savefig(fname, dpi=300)
    plt.close('all')

### Create Wheeler diagram

In [129]:
strat, wheeler, wheeler_strat, vacuity = sg.create_wheeler_diagram(topo, 0.5)

### Create 3D facies array

In [130]:
# create 3D facies array (as a function of water depth in this case)
ny, nx, nz = np.shape(strat)
facies = np.zeros((ny, nx, nz))
for i in range(facies.shape[2]):
    topo_sl = topo[:, :, i] - sea_level[i]
    facies_sl = np.zeros(np.shape(topo_sl))
    facies_sl[topo_sl >= 0] = 0
    facies_sl[(topo_sl < 0) & (topo_sl >= -100)] = 1
    facies_sl[topo_sl < -100] = 2
    facies[:, :, i] = facies_sl

In [8]:
# plot cross section colored by water depth (= facies in this case)
fig = sg.plot_model_cross_section_EW(strat, facies, facies, dx = 5, xsec = 265, 
                                     color_mode = 'facies', 
                                     map_aspect = 1,
                                     line_freq = 1,
                                     flattening_ind = False, units = 'mm')
plt.plot([0, strat.shape[1]*5], [sea_level[-1], sea_level[-1]]);

100%|█████████████████████████████████████████| 100/100 [00:14<00:00,  6.79it/s]


### Figure 22: Exploded view, colored by bathymetry

The photos used here (and in the next code cell) are available from: https://www.dropbox.com/scl/fo/8nv6jp5c8bzpt6lz117iy/h?rlkey=riey19hmu46dly8vgpikm3i82&dl=0

In [12]:
from skimage.transform import resize

end_time = 86
dx = 5.0
dip1 = 10
dip2 = 505
strike1 = 0
strike2 = 400

strat, wheeler, wheeler_strat, vacuity = sg.create_wheeler_diagram(topo[dip1:dip2, strike1:strike2, :end_time], 0.5)
bottom = np.min(strat) - 20

ts = end_time-1 
if ts*3 < 100:
    fno = '0'+str(ts*3)
else:
    fno = str(ts*3)
im = plt.imread('/Users/zoltan/Dropbox/Chronostratigraphy/Tulane/HighRes/TDWB_17_' + fno +'.jpg')
temp = np.zeros((5245-245+113, 3112-184+17, 3)).astype('uint8')
temp[:-113, :-17, :] = im[245:, 184:, :]
temp = resize(temp, (923, 531))*255
im = temp.astype('uint8')
im = np.swapaxes(im,0,1)
inds = np.indices(np.shape(topo[:,:,0]))
inds2 = np.argwhere(inds[0] < -0.65*inds[1] + 148)
inds3 = np.argwhere(inds[0] > 0.58*inds[1] + 360)
im[inds2[:,0], inds2[:,1], :] = 0
im[inds3[:,0], inds3[:,1], :] = 0
im = im[dip1:dip2, strike1:strike2, :]
model, kmeans_colors = sg.pick_colors(im, 256)
im_pred = model.predict(im.reshape(-1, 3))
im_pred = im_pred.reshape(im.shape[0], im.shape[1])

mlab.figure(bgcolor=(1,1,1))
# mlab.clf()

sg.create_exploded_view(topo[dip1:dip2, strike1:strike2, :end_time], strat, nx=3, ny=3, gap=50, dx=dx, ve=2, 
    color_mode='bathymetry', linewidth=2, bottom=bottom, opacity=1.0, x0=0, y0=0, texture=im_pred, 
    sea_level=sea_level, scale=1, plot_sides=True, plot_water=True, plot_surf=True, 
    surf_cmap='Blues', kmeans_colors=kmeans_colors, line_freq=1)

mlab.view(azimuth=-49, elevation=53, distance=4600, 
  focalpoint=np.array([ 1025,  1175, -2710]))

(-49.0, 53.0, 4600, array([ 1320.76344,  1485.     , -1326.12752]))

### Figure 21: Block diagram with arbitrarily oriented sections

In [15]:
from matplotlib import colors
# mlab.figure(bgcolor=(1,1,1))
mlab.clf()
ve = 3
dip1 = 10
dip2 = 505
strike1 = 0
strike2 = 400
end_time = 101

strat, wheeler, wheeler_strat, vacuity = sg.create_wheeler_diagram(topo[dip1:dip2, strike1:strike2, :end_time], 0.5)
bottom = np.min(strat) - 20

x1 = 0
y1 = 142
x2 = 225
y2 = 0
s1 = 0
sg.plot_random_section_2_points(topo[dip1:dip2, strike1:strike2, :end_time], strat, dx, x1, x2, y1, y2, s1, ve, 
    bottom = bottom, xoffset=0, yoffset=0, sea_level=sea_level, 
    linewidth=0.5, line_freq=1, color_mode='bathymetry', plot_type='3D', plot_basement=True)

x1 = 0
y1 = 348
x2 = 265
y2 = 494
s1 = 0
sg.plot_random_section_2_points(topo[dip1:dip2, strike1:strike2, :end_time], strat, dx, x1, x2, y1, y2, s1, ve, 
    bottom = bottom, xoffset=0, yoffset=0, sea_level=sea_level, 
    linewidth=0.5, line_freq=1, color_mode='bathymetry', plot_type='3D', plot_basement=True)

x1 = 0
y1 = 142
x2 = 0
y2 = 348
s1 = 0
sg.plot_random_section_2_points(topo[dip1:dip2, strike1:strike2, :end_time], strat, dx, x1, x2, y1, y2, s1, ve, 
    bottom = bottom, xoffset=0, yoffset=0, sea_level=sea_level, 
    linewidth=0.5, line_freq=1, color_mode='bathymetry', plot_type='3D', plot_basement=True)

x1 = 225
y1 = 0
x2 = 399
y2 = 0
s1 = 0
sg.plot_random_section_2_points(topo[dip1:dip2, strike1:strike2, :end_time], strat, dx, x1, x2, y1, y2, s1, ve, 
    bottom = bottom, xoffset=0, yoffset=0, sea_level=sea_level, 
    linewidth=0.5, line_freq=1, color_mode='bathymetry', plot_type='3D', plot_basement=True)

x1 = 265
y1 = 494
x2 = 399
y2 = 494
s1 = 0
sg.plot_random_section_2_points(topo[dip1:dip2, strike1:strike2, :end_time], strat, dx, x1, x2, y1, y2, s1, ve, 
    bottom = bottom, xoffset=0, yoffset=0, sea_level=sea_level, 
    linewidth=0.5, line_freq=1, color_mode='bathymetry', plot_type='3D', plot_basement=True)

x1 = 399
y1 = 0
x2 = 399
y2 = 494
s1 = 0
sg.plot_random_section_2_points(topo[dip1:dip2, strike1:strike2, :end_time], strat, dx, x1, x2, y1, y2, s1, ve, 
    bottom = bottom, xoffset=0, yoffset=0, sea_level=sea_level, 
    linewidth=0.5, line_freq=1, color_mode='bathymetry', plot_type='3D', plot_basement=True)

ts = end_time-1 
if ts*3 < 100:
    fno = '0'+str(ts*3)
else:
    fno = str(ts*3)
im = plt.imread('/Users/zoltan/Dropbox/Chronostratigraphy/Tulane/HighRes/TDWB_17_' + fno +'.jpg')
temp = np.zeros((5245-245+113, 3112-184+17, 3)).astype('uint8')
temp[:-113, :-17, :] = im[245:, 184:, :]
temp = resize(temp, (923, 531))*255
im = temp.astype('uint8')
im = np.swapaxes(im,0,1)
inds = np.indices(np.shape(topo[:,:,0]))
inds2 = np.argwhere(inds[0] < -0.65*inds[1] + 148)
inds3 = np.argwhere(inds[0] > 0.58*inds[1] + 360)
im[inds2[:,0], inds2[:,1], :] = 0
im[inds3[:,0], inds3[:,1], :] = 0
im = im[dip1:dip2, strike1:strike2, :]
model, kmeans_colors = sg.pick_colors(im, 256)
im_pred = model.predict(im.reshape(-1, 3))
im_pred = im_pred.reshape(im.shape[0], im.shape[1])

temp = np.zeros(np.shape(topo[dip1:dip2, strike1:strike2, end_time-1]))
inds = np.indices(np.shape(topo[dip1:dip2, strike1:strike2, end_time-1]))
x1 = 0
y1 = 348
x2 = 265
y2 = 494
slope, y_intercept = sg.line_coefficients(x1, y1, x2, y2)
inds2 = np.argwhere(inds[0] > slope*inds[1] + y_intercept)
x1 = 0
y1 = 142
x2 = 225
y2 = 0
slope, y_intercept = sg.line_coefficients(x1, y1, x2, y2)
inds3 = np.argwhere(inds[0] < slope*inds[1] + y_intercept)
temp[inds2[:,0], inds2[:,1]] = 1
temp[inds3[:,0], inds3[:,1]] = 1

sg.plot_surf_w_texture(strat, 1, dx, ve, im_pred.astype('float'), opacity=1.0, cmap='Blues', 
          mask = temp.astype('bool'), kmeans_colors=kmeans_colors)

# plot bottom face of block:
r,c,ts = np.shape(strat)
z = bottom*np.ones(np.shape(strat[:,:,0]))
z[temp==1] = np.nan
scale = 1
X1 = scale*(np.linspace(0,c-1,c)*dx) # x goes with c and y with r
Y1 = scale*(np.linspace(0,r-1,r)*dx)
X1_grid , Y1_grid = np.meshgrid(X1, Y1)
surf = mlab.mesh(X1_grid, Y1_grid, z*ve, scalars = z, mask = temp.astype('bool'), colormap='Blues', vmin=0, vmax=255)
lut = surf.module_manager.scalar_lut_manager.lut.table.to_array()
gray = np.expand_dims(colors.to_rgba('lightgray')[:3], axis=0)
lut[:,:3] = 255*np.repeat(gray, 256, axis=0)
surf.module_manager.scalar_lut_manager.lut.table = lut

# plot water surface:
# sl = sea_level[end_time-1]
# z = sl*np.ones(np.shape(strat[:,:,0]))
# z[z < strat[:, :, -1]] = np.nan
# z[temp==1] = np.nan
# X1 = scale*(np.linspace(0,c-1,c)*dx) # x goes with c and y with r
# Y1 = scale*(np.linspace(0,r-1,r)*dx)
# X1_grid , Y1_grid = np.meshgrid(X1, Y1)
# surf = mlab.mesh(X1_grid, Y1_grid, z*ve, scalars = np.zeros(np.shape(z)), mask = np.isnan(z), 
#                  colormap='Blues', vmin=0, vmax=255)
# lut = surf.module_manager.scalar_lut_manager.lut.table.to_array()
# blue = np.expand_dims(np.array([0.255, 0.412, 0.882]), axis=0)
# lut[:,:3] = 255*np.repeat(blue, 256, axis=0)
# lut[:,-1] = 255*0.3
# surf.module_manager.scalar_lut_manager.lut.table = lut

mlab.view(azimuth=-148, elevation=47, distance=4000, 
  focalpoint=np.array([ 950,  1020, -3762]))

100%|█████████████████████████████████████████| 100/100 [00:04<00:00, 24.29it/s]


(-148.0, 47.0, 4000, array([  997.5  ,  1235.   , -3502.878]))

### Strike and dip sections, colored by rate of sea level change

In [17]:
# you have to use 'sea_level_rel' instaed of 'sea_level'
sea_level_rel = sea_level - (sea_level[0]+np.cumsum(0.75*np.ones(np.shape(sea_level)))-0.75)
sg.plot_strike_section(topo[dip1:dip2, :500, :], sg.topostrat(topo[dip1:dip2, :500, :]), 
                    dx, 150, ve=ve, sea_level = sea_level_rel, 
                    linewidth=0.5, line_freq=1, 
                    color_mode='sea_level_change', plot_type='2D', 
                    plot_erosion=False, plot_water=True, plot_basement=True)
sg.plot_dip_section(topo[dip1:dip2, :500, :], sg.topostrat(topo[dip1:dip2, :500, :]), 
                    dx, 150, ve=ve, sea_level = sea_level_rel, 
                    linewidth=0.5, line_freq=1, 
                    color_mode='sea_level_change', plot_type='2D', 
                    plot_erosion=False, plot_water=True, plot_basement=True)

### Strike and dip sections, colored by age

In [19]:
sg.plot_strike_section(topo[dip1:dip2, :500, :], sg.topostrat(topo[dip1:dip2, :500, :]), 
                    dx, 150, ve=ve, sea_level = sea_level, 
                    linewidth=0.5, line_freq=1, 
                    color_mode='age', plot_type='2D', 
                    plot_erosion=False, plot_water=True, plot_basement=True)
sg.plot_dip_section(topo[dip1:dip2, :500, :], sg.topostrat(topo[dip1:dip2, :500, :]), 
                    dx, 150, ve=ve, sea_level = sea_level, 
                    linewidth=0.5, line_freq=1, 
                    color_mode='age', plot_type='2D', 
                    plot_erosion=False, plot_water=True, plot_basement=True)

### Strike and dip sections, colored by bathymetry

In [22]:
sg.plot_strike_section(topo[dip1:dip2, :500, :], sg.topostrat(topo[dip1:dip2, :500, :]), 
                    dx, 150, ve=ve, sea_level = sea_level, 
                    linewidth=0.5, line_freq=1, 
                    color_mode='bathymetry', plot_type='2D', 
                    plot_erosion=False, plot_water=True, plot_basement=True)
sg.plot_dip_section(topo[dip1:dip2, :500, :], sg.topostrat(topo[dip1:dip2, :500, :]), 
                    dx, 150, ve=ve, sea_level = sea_level, 
                    linewidth=0.5, line_freq=1, 
                    color_mode='bathymetry', plot_type='2D', 
                    plot_erosion=False, plot_water=True, plot_basement=True)

### Fence diagram

In [23]:
mlab.clf()
end_time = 86
strat, wheeler, wheeler_strat, vacuity = sg.create_wheeler_diagram(topo[dip1:dip2, \
                                                        strike1:strike2, :end_time], 0.5)

ts = end_time-1 
if ts*3 < 100:
    fno = '0'+str(ts*3)
else:
    fno = str(ts*3)
im = plt.imread('/Users/zoltan/Dropbox/Chronostratigraphy/Tulane/HighRes/TDWB_17_' + fno +'.jpg')
temp = np.zeros((5245-245+113, 3112-184+17, 3)).astype('uint8')
temp[:-113, :-17, :] = im[245:, 184:, :]
temp = resize(temp, (923, 531))*255
im = temp.astype('uint8')
im = np.swapaxes(im,0,1)
inds = np.indices(np.shape(topo[:,:,0]))
inds2 = np.argwhere(inds[0] < -0.65*inds[1] + 148)
inds3 = np.argwhere(inds[0] > 0.58*inds[1] + 360)
im[inds2[:,0], inds2[:,1], :] = 0
im[inds3[:,0], inds3[:,1], :] = 0
im = im[dip1:dip2, strike1:strike2, :]
model, kmeans_colors = sg.pick_colors(im, 256)
im_pred = model.predict(im.reshape(-1, 3))
im_pred = im_pred.reshape(im.shape[0], im.shape[1])

sg.create_fence_diagram(topo, strat, nx=3, ny=3, dx=dx, ve=2, 
    color_mode='bathymetry', sea_level=sea_level, linewidth=0.5, bottom=bottom, opacity=0.7, 
    scale=1, plot_sides=True, plot_water=False, plot_surf=True,
    surf_cmap='Blues', texture=im_pred, kmeans_colors=kmeans_colors, line_freq=1)

100%|█████████████████████████████████████████████| 5/5 [00:14<00:00,  2.84s/it]


(array([  0, 100, 200, 300, 399]), array([  0, 123, 246, 369, 494]))

### Exploded view, 4 dip-oriented blocks

In [24]:
mlab.clf()
dx = 5.0
dip1 = 10+70
dip2 = 505-70
strike1 = 0
strike2 = 450
end_time = 101

strat, wheeler, wheeler_strat, vacuity = sg.create_wheeler_diagram(topo[dip1:dip2, \
                                                    strike1:strike2, :end_time], 0.5)
bottom = np.min(strat) - 20
ts = end_time-1
if ts*3 < 10:
    fno = '00'+str(ts*3)
elif 10 <= ts*3 < 100:
    fno = '0'+str(ts*3)
else:
    fno = str(ts*3)
im = plt.imread('/Users/zoltan/Dropbox/Chronostratigraphy/Tulane/HighRes/TDWB_17_' + fno +'.jpg')
temp = np.zeros((5245-245+113, 3112-184+17, 3)).astype('uint8')
temp[:-113, :-17, :] = im[245:, 184:, :]
temp = resize(temp, (923, 531))*255
im = temp.astype('uint8')
im = np.swapaxes(im,0,1)
inds = np.indices(np.shape(topo[:,:,0]))
inds2 = np.argwhere(inds[0] < -0.65*inds[1] + 148)
inds3 = np.argwhere(inds[0] > 0.58*inds[1] + 360)
im[inds2[:,0], inds2[:,1], :] = 0
im[inds3[:,0], inds3[:,1], :] = 0
im = im[dip1:dip2, strike1:strike2, :]
model, kmeans_colors = sg.pick_colors(im, 256)
im_pred = model.predict(im.reshape(-1, 3))
im_pred = im_pred.reshape(im.shape[0], im.shape[1])
gap = 50
sg.create_exploded_view(topo[dip1:dip2, strike1:strike2, :end_time], strat, nx=1, ny=4, gap=gap, dx=dx, ve=3, 
    color_mode='bathymetry', linewidth=2, bottom=bottom, opacity=1.0, x0=0, y0=-5*gap*dx, texture=im_pred, 
    sea_level=sea_level, scale=1, plot_sides=True, plot_water=True, plot_surf=True, 
    surf_cmap='Blues', kmeans_colors=kmeans_colors, line_freq=1, water_depth=50)

mlab.view(azimuth=-50, elevation=50, distance=6000, 
  focalpoint=np.array([ 993,  41.4, -4067]))

(-50.0, 50.0, 6000, array([ 2023.48026,     2.5    , -2031.2049 ]))

In [133]:
dx

1

### Figures 25, 26: Plot strike section and Wheeler diagram

In [26]:
from importlib import reload
reload(sg)

<module 'stratigraph' from '/Users/zoltan/Dropbox/Chronostratigraphy/stratigraph/stratigraph/stratigraph.py'>

In [32]:
# need to recompute Wheeler diagram and stratigraphy:
strat, wheeler, wheeler_strat, vacuity = sg.create_wheeler_diagram(topo, 0.5)

In [131]:
# compute erosional surface attributes
time = scan_times
erosional_surfs_age_below, erosional_surfs_age_above, erosional_surfs_time, erosional_surfs_thickness =\
        sg.compute_erosional_surf_attributes(strat, time, topo, erosion_threshold=0.1)

100%|█████████████████████████████████████████| 531/531 [02:04<00:00,  4.28it/s]


In [34]:
# strike section
reload(sg)

rdbu = mpl.colormaps['RdBu'].resampled(256)
newcolors = rdbu(np.linspace(0, 1, 256))
newcolors[126:131, :] = np.array([1, 1, 1, 1])
newcmp = ListedColormap(newcolors)

# loc = 100 # Figure 25
loc = 200 # Figure 26
dx = 5.0
fig = plt.figure(figsize=(15,15))
spec = gridspec.GridSpec(ncols=3, nrows=2,
                         width_ratios=[1, 6, 0.2], wspace=0.04,
                         hspace=0.05, height_ratios=[3, 2])
ax0 = fig.add_subplot(spec[1,0])
ax1 = fig.add_subplot(spec[1,1], sharey = ax0)
ax2 = fig.add_subplot(spec[0,1], sharex = ax1)
ax3 = fig.add_subplot(spec[1,2])
im = ax1.imshow(wheeler[:, loc, :].T, cmap=newcmp, vmin = -10, vmax = 10, extent = [0, dx*strat.shape[0], 
        scan_times[-1], 0], interpolation='none', aspect='auto')
ax1.invert_yaxis()
ax1.set_xlabel('distance (mm)', fontsize = 14)
ax0.set_ylabel('time (days)', fontsize = 14)
plt.setp(ax1.get_yticklabels(), visible=False)
plt.setp(ax2.get_xticklabels(), visible=False)

plt.colorbar(im, cax=ax3);
# ax0.plot(sea_level, scan_times, 'k', linewidth=3)
cmap = mpl.colormaps['viridis']
norm = mpl.colors.Normalize(vmin=0, vmax=len(scan_times)-1)
for i in range(len(scan_times)-1):
    ax0.plot([sea_level[i], sea_level[i+1]], [scan_times[i], scan_times[i+1]], 
             color=cmap(norm(i))[:3], linewidth=3)

ax1.set_xlim(0, 2500)
ax2.set_xlim(0, 2465)
ax2.set_ylim(-1225, -1050)

sg.plot_strike_section(topo[:,:,:end_time], sg.topostrat(topo[:,:,:end_time]), 
                    dx, loc, ve, ax=ax2, sea_level = sea_level[:end_time], 
                    linewidth=0.5, line_freq=1, 
                    color_mode='bathymetry', plot_type='2D', 
                    plot_erosion=True, erosional_surfs_thickness=erosional_surfs_thickness,
                    plot_water=True, plot_basement=True)

### Figures 23, 24: Plot dip section and Wheeler diagram

In [134]:
# dip section
rdbu = mpl.colormaps['RdBu'].resampled(256)
newcolors = rdbu(np.linspace(0, 1, 256))
newcolors[126:131, :] = np.array([1, 1, 1, 1])
newcmp = ListedColormap(newcolors)
reload(sg)
from matplotlib import gridspec

loc = 251 # Figure 24
# loc = 220 # Figure 23
dx = 5.0

fig = plt.figure(figsize=(15,15))
spec = gridspec.GridSpec(ncols=3, nrows=2,
                         width_ratios=[1, 6, 0.2], wspace=0.04,
                         hspace=0.05, height_ratios=[3, 2])
ax0 = fig.add_subplot(spec[1,0])
ax1 = fig.add_subplot(spec[1,1], sharey = ax0)
ax2 = fig.add_subplot(spec[0,1], sharex = ax1)
ax3 = fig.add_subplot(spec[1,2])
im = ax1.imshow(wheeler[loc, :, :].T, cmap=newcmp, vmin = -10, vmax = 10, extent = [0, dx*strat.shape[1], 
        scan_times[-1], 0], interpolation='none', aspect='auto')
ax1.invert_yaxis()
ax1.set_xlabel('distance (mm)', fontsize = 14)
ax0.set_ylabel('time (days)', fontsize = 14)
plt.setp(ax1.get_yticklabels(), visible=False)
plt.setp(ax2.get_xticklabels(), visible=False)

plt.colorbar(im, cax=ax3);
# ax0.plot(sea_level, scan_times, 'k', linewidth=3)
cmap = mpl.colormaps['viridis']
norm = mpl.colors.Normalize(vmin=0, vmax=len(scan_times)-1)
for i in range(len(scan_times)-1):
    ax0.plot([sea_level[i], sea_level[i+1]], [scan_times[i], scan_times[i+1]], 
             color=cmap(norm(i))[:3], linewidth=3)

ax1.set_xlim(0, 2500)
ax2.set_ylim(-1380, -1050)
sg.plot_dip_section(topo[:,:,:end_time], sg.topostrat(topo[:,:,:end_time]), 
                    dx, loc, ve, ax=ax2, sea_level = sea_level[:end_time], 
                    linewidth=0.5, line_freq=1, 
                    color_mode='bathymetry', plot_type='2D', 
                    plot_erosion=True, erosional_surfs_thickness=erosional_surfs_thickness,
                    plot_water=True, plot_basement=True)

In [155]:
loc = 251 # Figure 24
# loc = 220 # Figure 23
dx = 5.0

for end_time in range(101, 102): #, 2):
    strat_temp, wheeler_temp, wheeler_strat_temp, vacuity_temp = sg.create_wheeler_diagram(topo[:,:,:end_time], 0.5)
    erosional_surfs_age_below, erosional_surfs_age_above, erosional_surfs_time, erosional_surfs_thickness =\
        sg.compute_erosional_surf_attributes(strat_temp, scan_times[:end_time], topo[:,:,:end_time])
    
    fig = plt.figure(figsize=(15,15))
    spec = gridspec.GridSpec(ncols=3, nrows=2,
                             width_ratios=[1, 6, 0.2], wspace=0.04,
                             hspace=0.05, height_ratios=[3, 2])
    ax0 = fig.add_subplot(spec[1,0])
    ax0.set_xlim(-1162, -1080)
    ax1 = fig.add_subplot(spec[1,1], sharey = ax0)
    ax2 = fig.add_subplot(spec[0,1], sharex = ax1)
    ax3 = fig.add_subplot(spec[1,2])
    im = ax1.imshow(wheeler_temp[loc, :, :].T, cmap=newcmp, vmin = -10, vmax = 10, 
                    extent = [0, dx*strat.shape[1], scan_times[:end_time][-1], 0], interpolation='none', aspect='auto')
    ax1.invert_yaxis()
    ax1.set_ylim(0, scan_times[-1])
    ax1.set_xlabel('distance (mm)', fontsize = 14)
    ax0.set_ylabel('time (days)', fontsize = 14)
    plt.setp(ax1.get_yticklabels(), visible=False)
    plt.setp(ax2.get_xticklabels(), visible=False)

    plt.colorbar(im, cax=ax3);
    # ax0.plot(sea_level, scan_times, 'k', linewidth=3)
    cmap = mpl.colormaps['viridis']
    norm = mpl.colors.Normalize(vmin=0, vmax=len(scan_times)-1)
    for i in range(len(scan_times[:end_time])-1):
        ax0.plot([sea_level[i], sea_level[i+1]], [scan_times[i], scan_times[i+1]], 
                 color=cmap(norm(i))[:3], linewidth=3)

    ax1.set_xlim(0, 2500)
    ax2.set_ylim(-1380, -1050)
    sg.plot_dip_section(topo[:,:,:end_time], strat_temp, 
                        dx, loc, ve, ax=ax2, sea_level = sea_level[:end_time], 
                        linewidth=0.5, line_freq=1, 
                        color_mode='bathymetry', plot_type='2D', 
                        plot_erosion=True, erosional_surfs_thickness=erosional_surfs_thickness,
                        plot_water=True, plot_basement=True)
    
    fname = '/Users/zoltan/Dropbox/Chronostratigraphy/movie_frames/TDWB_17_dip_section_251_'+'%03d.png'%(51)
    fig.savefig(fname, dpi=300)
    plt.close('all')

### Arbitrary section between two points

In [37]:
from scipy.ndimage import map_coordinates

x1 = 40
y1 = 245
x2 = 356
y2 = 27

# x1 = 100
# y1 = 174
# x2 = 122
# y2 = 215

# x1 = 28
# y1 = 307
# x2 = 300
# y2 = 177

# x1 = 15
# y1 = 258
# x2 = 423
# y2 = 247

# x1 = 150
# y1 = 380
# x2 = 451
# y2 = 355

s1 = 0

end_time = 101
strat, wheeler, wheeler_strat, vacuity = sg.create_wheeler_diagram(topo[:,:,:end_time], 0.5)

rdbu = mpl.colormaps['RdBu'].resampled(256)
newcolors = rdbu(np.linspace(0, 1, 256))
newcolors[126:131, :] = np.array([1, 1, 1, 1])
newcmp = ListedColormap(newcolors)
reload(sg)
from matplotlib import gridspec

fig = plt.figure(figsize=(15,15))
spec = gridspec.GridSpec(ncols=3, nrows=2,
                         width_ratios=[1, 6, 0.5], wspace=0.0,
                         hspace=0.0, height_ratios=[1, 1])
ax0 = fig.add_subplot(spec[1,0])
ax1 = fig.add_subplot(spec[1,1], sharey = ax0)
ax2 = fig.add_subplot(spec[0,1], sharex = ax1)
ax3 = fig.add_subplot(spec[1,2])

dist = dx*((x2-x1)**2 + (y2-y1)**2)**0.5
s2 = s1*dx+dist
num = int(dist/float(dx))
Xrand, Yrand, Srand = np.linspace(x1,x2,num), np.linspace(y1,y2,num), np.linspace(s1*dx,s2,num)

wheeler_section = np.zeros((len(Srand), wheeler.shape[2]))
for layer_n in trange(0, wheeler.shape[2]):
    wheeler_section[:, layer_n] = map_coordinates(wheeler[:,:,layer_n], np.vstack((Yrand,Xrand)))

im = ax1.imshow(wheeler_section.T, cmap=newcmp, vmin = -10, vmax = 10, extent = [0, Srand[-1], 
        scan_times[end_time-1], 0], interpolation='none', aspect='auto')
ax1.invert_yaxis()
ax1.set_xlabel('distance (mm)', fontsize = 14)
ax1.set_ylabel('time (days)', fontsize = 14)
plt.setp(ax1.get_yticklabels(), visible=False)
plt.setp(ax2.get_xticklabels(), visible=False)
ax3.set_axis_off()

plt.colorbar(im, ax=ax3);
ax0.plot(sea_level[:end_time], scan_times[:end_time], 'k', linewidth=3)

sg.plot_random_section_2_points(topo[:, :, :end_time], strat, dx, x1, x2, y1, y2, s1, ve, ax=ax2,
    bottom = -1350, xoffset=0, yoffset=0, sea_level=sea_level[:end_time], 
    linewidth=0.5, line_freq=1, color_mode='bathymetry', plot_type='2D', plot_basement=True, plot_erosion=True,
                               erosional_surfs_thickness = erosional_surfs_thickness)

100%|█████████████████████████████████████████| 100/100 [00:07<00:00, 14.19it/s]


### Figure 27: Time-slice through Wheeler diagram

In [47]:
def plot_time_step(strat, wheeler, topo, dx, sea_level, ts, cmap, fig, ax, shrink):
    im = ax.imshow(wheeler[::-1, :, ts-1], cmap=cmap, vmin = -10, vmax = 10, 
               extent = [0, dx*strat.shape[1], 0, dx*strat.shape[0]], interpolation='none')
    x, y = np.meshgrid(np.arange(0, dx*strat.shape[1], dx), np.arange(0, dx*strat.shape[0], dx))
    temp = sg.sgolay2d(topo[:, :, ts], 5, 3)
    ax.contour(x, y, temp, colors='k', linewidths=0.2, linestyles ='solid', levels=np.arange(-1600,-960,5))
    ax.contour(x, y, temp, colors='k', linewidths=2, linestyles ='dashed', 
                levels=[sea_level[ts]])
    ax.set_xlim(0, 2500)
    ax.set_ylim(0, 2560)
    fig.colorbar(im, ax=ax, shrink=shrink, pad = 0.02)
    
rdbu = mpl.colormaps['RdBu'].resampled(256)
newcolors = rdbu(np.linspace(0, 1, 256))
newcolors[126:131, :] = np.array([1, 1, 1, 1])
rdbucmp = ListedColormap(newcolors)

fig, axes = plt.subplots(3, 2, figsize=(8, 10))
count = 0
for ts in [81, 85, 89, 93, 97, 100]: #range(85, 85+3*6, 3):
    plot_time_step(strat, wheeler, topo, dx, sea_level, ts, rdbucmp, fig, axes[count//2, count%2], shrink=0.97)
    if ts == 85:
        axes[count//2, count%2].plot(50*dx, 244*dx, 'ro')
        axes[count//2, count%2].plot(145*dx, 251*dx, 'ro')
        axes[count//2, count%2].plot(215*dx, 129*dx, 'ro')
        axes[count//2, count%2].plot(324*dx, 249*dx, 'ro')
    count += 1

### 3D visualiztion with plane widgets

In [40]:
mlab.clf()
source = mlab.pipeline.scalar_field(np.swapaxes(wheeler, 0, 1))
source.spacing = [1,1,3]
for axis in ['x', 'y', 'z']:
    plane = mlab.pipeline.image_plane_widget(source, plane_orientation = '{}_axes'.format(axis),
                                           slice_index=i, colormap='RdBu', vmin = -6, vmax = 6);

### Figure 28: Maps of stratigraphic attributes

In [41]:
deposition_time, erosion_time, stasis_time, vacuity_time, deposition_thickness, erosion_thickness =\
        sg.compute_strat_maps(strat, wheeler, wheeler_strat, vacuity)

In [42]:
# set 'uninteresting' areas to NaN:
deposition_time[stasis_time > 0.98] = np.nan
erosion_time[stasis_time > 0.98] = np.nan
vacuity_time[stasis_time > 0.98] = np.nan
deposition_thickness[stasis_time > 0.98] = np.nan
erosion_thickness[stasis_time > 0.98] = np.nan
stasis_time[stasis_time > 0.98] = np.nan

# smooth maps:
deposition_time = sg.smooth_strat_attribute(deposition_time, 11)
erosion_time = sg.smooth_strat_attribute(erosion_time, 21)
stasis_time = sg.smooth_strat_attribute(stasis_time, 11)
vacuity_time = sg.smooth_strat_attribute(vacuity_time, 21)
deposition_thickness = sg.smooth_strat_attribute(deposition_thickness, 11)
erosion_thickness = sg.smooth_strat_attribute(erosion_thickness, 21)

In [43]:
# clip colormap (so that colors are not too dark at the lower end)
import cmocean
cmap = cmocean.cm.deep_r
newcmap = cmocean.tools.crop_by_percent(cmap, 20, which='min', N=None)

fig, axs = plt.subplots(2, 3, sharey=True, figsize=(20, 8))
im = axs[0,0].imshow(deposition_time[::-1,:], vmin=0, vmax=0.5, cmap=newcmap)
axs[0,0].contour(deposition_time[::-1,:], levels=np.linspace(0,0.5,10), colors='k', linewidths=0.5)
axs[0,0].set_title('deposition (time)')
# axs[0,0].set_xticks([])
# axs[0,0].set_yticks([])
axs[0,0].set_ylim(520, 22)
fig.colorbar(im, ax=axs[0,0], shrink=0.65)
im = axs[0,1].imshow(erosion_time[::-1,:], vmin=0, vmax=0.2, cmap=newcmap)
axs[0,1].contour(erosion_time[::-1,:], levels=np.linspace(0,0.2,10), colors='k', linewidths=0.5)
axs[0,1].set_title('erosion (time)')
axs[0,1].set_xticks([])
axs[0,1].set_yticks([])
fig.colorbar(im, ax=axs[0,1], shrink=0.65)
im = axs[0,2].imshow(stasis_time[::-1,:], vmin=0, vmax=1, cmap=newcmap)
axs[0,2].contour(stasis_time[::-1,:], levels=np.linspace(0,1,10), colors='k', linewidths=0.5)
axs[0,2].set_title('stasis (time)')
axs[0,2].set_xticks([])
axs[0,2].set_yticks([])
fig.colorbar(im, ax=axs[0,2], shrink=0.65)
im = axs[1,0].imshow(vacuity_time[::-1,:], vmin=0, vmax=0.3, cmap=newcmap)
axs[1,0].contour(vacuity_time[::-1,:], levels=np.linspace(0,0.3,10), colors='k', linewidths=0.5)
axs[1,0].set_title('vacuity (time)')
axs[1,0].set_xticks([])
axs[1,0].set_yticks([])
fig.colorbar(im, ax=axs[1,0], shrink=0.65)
im = axs[1,1].imshow(deposition_thickness[::-1,:], vmin=10, vmax=150, cmap=newcmap)
axs[1,1].contour(deposition_thickness[::-1,:], levels=np.linspace(10,150,20), colors='k', linewidths=0.5)
axs[1,1].set_title('deposition (thickness)')
axs[1,1].set_xticks([])
axs[1,1].set_yticks([])
fig.colorbar(im, ax=axs[1,1], shrink=0.65)
im = axs[1,2].imshow(-erosion_thickness[::-1,:], vmin=0, vmax=60, cmap=newcmap)
axs[1,2].contour(-erosion_thickness[::-1,:], levels=np.linspace(0,60,20), colors='k', linewidths=0.5)
axs[1,2].set_title('erosion (thickness)')
axs[1,2].set_xticks([])
axs[1,2].set_yticks([])
fig.colorbar(im, ax=axs[1,2], shrink=0.65)
fig.tight_layout()

### Plot erosional attributes of stratigraphic surfaces

In [48]:
import cmocean
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, InsetPosition
from matplotlib.gridspec import GridSpec

cmap = cmocean.cm.deep_r
newcmap = cmocean.tools.crop_by_percent(cmap, 30, which='min', N=None)

def plot_strat_surface_attributes(erosional_surfs_first_age, erosional_surfs_last_age, 
                                  erosional_surfs_time, erosional_surfs_thickness, ts, cmap):
    fig = plt.figure(figsize = (10, 10))
    gs = GridSpec(2, 2, left=0.05, right=0.98, wspace=0.001)
    
    ax1 = fig.add_subplot(gs[0, 0])
    temp = erosional_surfs_first_age[:, :, ts].copy()
    temp[temp==-1] = np.nan
    im = ax1.imshow(temp*0.125, aspect=1, interpolation='none', cmap=cmap, 
                          vmin=0, vmax=12.5)
    ax1.contour(strat[:,:,ts], colors='k', levels=100, linestyles='solid', linewidths=0.3)
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.set_title('age of deposits below')
    cbar = fig.colorbar(im, shrink=1, ax=ax1)
    cbar.ax.get_yaxis().labelpad = 15
    cbar.ax.set_ylabel('age (days)', rotation=270)
    ax1.set_xlim(0, 2500/5)
    ax1.set_ylim(0, 2560/5)
    
    ax2 = fig.add_subplot(gs[0, 1])
    temp = erosional_surfs_last_age[:, :, ts].copy()
    temp[temp==-1] = np.nan
    im = ax2.imshow(temp*0.125, aspect=1, interpolation='none', cmap=cmap, 
                          vmin=0, vmax=12.5)
    ax2.contour(strat[:,:,ts], colors='k', levels=100, linestyles='solid', linewidths=0.3)
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax2.set_title('age of deposits above')
    cbar = fig.colorbar(im, shrink=1, ax=ax2)
    cbar.ax.get_yaxis().labelpad = 15
    cbar.ax.set_ylabel('age (days)', rotation=270)
    ax2.set_xlim(0, 2500/5)
    ax2.set_ylim(0, 2560/5)
    
    ax3 = fig.add_subplot(gs[1, 0])
    temp = erosional_surfs_time[:, :, ts].copy()
    temp[temp==-1] = np.nan
    im = ax3.imshow(temp*0.125, aspect=1, interpolation='none', cmap=cmap, 
                          vmin=0, vmax=6)
    ax3.contour(strat[:,:,ts], colors='k', levels=100, linestyles='solid', linewidths=0.3)
    ax3.set_xticks([])
    ax3.set_yticks([])
    ax3.set_title('time gap')
    cbar = fig.colorbar(im, shrink=1, ax=ax3)
    cbar.ax.get_yaxis().labelpad = 15
    cbar.ax.set_ylabel('time gap (days)', rotation=270)
    ax3.set_xlim(0, 2500/5)
    ax3.set_ylim(0, 2560/5)
    
    ax4 = fig.add_subplot(gs[1, 1])
    temp = erosional_surfs_thickness[:, :, ts].copy()
    temp[temp==-1] = np.nan
    im = ax4.imshow(temp, aspect=1, interpolation='none', cmap=cmap, 
                          vmin=0, vmax=40)
    ax4.contour(strat[:,:,ts], colors='k', levels=100, linestyles='solid', linewidths=0.3)
    ax4.set_xticks([])
    ax4.set_yticks([])
    ax4.set_title('thickness eroded')
    cbar = fig.colorbar(im, shrink=1, ax=ax4)
    cbar.ax.get_yaxis().labelpad = 15
    cbar.ax.set_ylabel('thickness (mm)', rotation=270)
    ax4.set_xlim(0, 2500/5)
    ax4.set_ylim(0, 2560/5)
    
    return fig

ts = 85
fig = plot_strat_surface_attributes(erosional_surfs_age_below, erosional_surfs_age_above, 
                                  erosional_surfs_time, erosional_surfs_thickness, ts, newcmap)

## TDB-10-1

### Load data

The original, complete dataset for this experiment is available from:

[Wang, Y. and Straub, K.M., 2017, TDB_10_1, SEAD, DOI: 10.5967/M0HX19TT](http://sead-published.ncsa.illinois.edu/seadrepository/api/researchobjects/urn:uuid:59711af1e4b011ce953a57b8)

In [49]:
tdb_10_D = sio.loadmat('../data/Topo_TDB_10_1_D.mat')
tdb_10_M = sio.loadmat('../data/Topo_TDB_10_1_M.mat')
tdb_10_P = sio.loadmat('../data/Topo_TDB_10_1_P.mat')
topo_M = tdb_10_M['Topo_TDB_10_1_M']
topo_P = tdb_10_P['Topo_TDB_10_1_P']
topo_D = tdb_10_D['Topo_TDB_10_1_D']

In [50]:
# a bit of smoothing helps
from scipy.signal import savgol_filter
for i in range(topo_P.shape[-1]):
    topo_P[:, i] = savgol_filter(topo_P[:, i], 21, 2)
for i in range(topo_M.shape[-1]):
    topo_M[:, i] = savgol_filter(topo_M[:, i], 21, 2)
for i in range(topo_D.shape[-1]):
    topo_D[:, i] = savgol_filter(topo_D[:, i], 21, 2)

In [51]:
strat_M = sg.topostrat(topo_M)
strat_P = sg.topostrat(topo_P)
strat_D = sg.topostrat(topo_D)

### Figure 30C: Time-elevation plot

In [52]:
reload(sg)
elevation = topo_M[1000, :].copy()
time = np.arange(topo_M.shape[-1])*2  # units are minutes; scans were done every two minutes 
fig, dve_data, duration_thickness_data, ts_labels, strat_tops, strat_top_inds, strat_topbound_inds, interval_labels\
     = sg.plot_strat_diagram(elevation, 'mm', time, 'minutes', 0.5, 
                max_elevation=np.max(elevation), max_time=np.max(time), plotting=True, plot_raw_data=True)

### Figure 30A, B: Strike section and Wheeler diagram

In [70]:
reload(sg)
strat, wheeler, wheeler_strat, vacuity, stasis = sg.create_wheeler_diagram_2D(topo_M, 1.0)
erosional_surfs_age_below, erosional_surfs_age_above, erosional_surfs_time, erosional_surfs_thickness =\
        sg.compute_erosional_surf_attributes(strat, time, topo_M, erosion_threshold=0.1)

100%|██████████████████████████████████████| 2284/2284 [00:11<00:00, 204.16it/s]


In [71]:
strat, wheeler, wheeler_strat, vacuity, stasis = sg.create_wheeler_diagram_2D(topo_M[:,1500:2000], 1.0)
loc = 0
dx = 1
fig = plt.figure(figsize=(15,12))
spec = gridspec.GridSpec(ncols=3, nrows=2,
                         width_ratios=[1, 6, 0.2], wspace=0.04,
                         hspace=0.05, height_ratios=[3, 2])
ax0 = fig.add_subplot(spec[1,0])
ax1 = fig.add_subplot(spec[1,1], sharey = ax0)
ax2 = fig.add_subplot(spec[0,1], sharex = ax1)
ax3 = fig.add_subplot(spec[1,2])
im = ax1.imshow(wheeler_strat.T, cmap=newcmp, vmin = -5, vmax = 5, extent = [0, dx*(strat.shape[0]-1), 
        time[-1], 0], interpolation='none', aspect='auto')
ax1.invert_yaxis()
ax1.set_xlabel('distance (mm)', fontsize = 14)
ax2.set_ylabel('depth (mm)', fontsize = 14)
ax0.set_ylabel('time (minutes)', fontsize = 14)
ax0.set_xlabel('base level (mm)', fontsize = 14)
plt.setp(ax1.get_yticklabels(), visible=False)
plt.setp(ax2.get_xticklabels(), visible=False)
plt.colorbar(im, cax=ax3)
ax3.text(2.5, -8, 'erosion (mm)', rotation=90)
ax3.text(2.5, 1.8, 'deposition (mm)', rotation=90)
cmap = mpl.colormaps['viridis']
norm = mpl.colors.Normalize(vmin=0, vmax=len(time)-1)

    
topo = topo_M[:,1500:2000].copy()
topo = topo[:,np.newaxis,:]
er_surfs = erosional_surfs_thickness[:,1500:2000].copy()
er_surfs = er_surfs[:,np.newaxis,:]
sg.plot_strike_section(topo, sg.topostrat(topo), 
                    dx, loc, ve, ax=ax2, 
                    linewidth=0.25, line_freq=1, color_mode='age', plot_type='2D', 
                    plot_erosion=True, erosional_surfs_thickness=er_surfs, 
                    plot_water=False, plot_basement=True)

### Figure 31C: Time-elevation plot for downsampled surface data

In [74]:
reload(sg)
elevation = topo_M[1600, 1500:2000:10].copy()
fig, dve_data, duration_thickness_data, ts_labels, strat_tops, strat_top_inds, strat_topbound_inds, interval_labels\
 = sg.plot_strat_diagram(elevation, 'mm', time[1500:2000:10], 'minutes', 0.5, 
                max_elevation=np.max(elevation), max_time=np.max(time[1500:2000:10]), plotting=True, plot_raw_data=True)

### Figure 31A, B: Strike section and Wheeler diagram for downsampled surface data

In [76]:
strat, wheeler, wheeler_strat, vacuity, stasis = sg.create_wheeler_diagram_2D(topo_M[:,1500:2000:10], 1.0)
loc = 0
dx = 1
fig = plt.figure(figsize=(15,12))
spec = gridspec.GridSpec(ncols=3, nrows=2,
                         width_ratios=[1, 6, 0.2], wspace=0.04,
                         hspace=0.05, height_ratios=[3, 2])
ax0 = fig.add_subplot(spec[1,0])
ax1 = fig.add_subplot(spec[1,1], sharey = ax0)
ax2 = fig.add_subplot(spec[0,1], sharex = ax1)
ax3 = fig.add_subplot(spec[1,2])
im = ax1.imshow(wheeler_strat.T, cmap=newcmp, vmin = -5, vmax = 5, extent = [0, dx*(strat.shape[0]-1), 
        time[-1], 0], interpolation='none', aspect='auto')
ax1.invert_yaxis()
ax1.set_xlabel('distance (mm)', fontsize = 14)
ax2.set_ylabel('depth (mm)', fontsize = 14)
ax0.set_ylabel('time (minutes)', fontsize = 14)
ax0.set_xlabel('base level (mm)', fontsize = 14)
plt.setp(ax1.get_yticklabels(), visible=False)
plt.setp(ax2.get_xticklabels(), visible=False)
plt.colorbar(im, cax=ax3)
ax3.text(2.5, -8, 'erosion (mm)', rotation=90)
ax3.text(2.5, 1.8, 'deposition (mm)', rotation=90)
cmap = mpl.colormaps['viridis']
norm = mpl.colors.Normalize(vmin=0, vmax=len(time)-1)

    
topo = topo_M[:,1500:2000:10].copy()
topo = topo[:,np.newaxis,:]
er_surfs = erosional_surfs_thickness[:,1500:2000:10].copy()
er_surfs = er_surfs[:,np.newaxis,:]
sg.plot_strike_section(topo, sg.topostrat(topo), 
                    dx, loc, ve, ax=ax2, 
                    linewidth=0.25, line_freq=1, color_mode='age', plot_type='2D', 
                    plot_erosion=True, erosional_surfs_thickness=er_surfs, 
                    plot_water=False, plot_basement=True)